In [1]:
import numpy as np

from src.graphic_interface.simu_result import load_all_simulation
from src.surface.surface_Fourier import Surface_Fourier,plot,plot_function_on_surface
import src.surface.surface_Fourier,src.tools


In [2]:
column_labels=["Name","lambda","Penalization","cost B","max B","cost j","max j","EM cost","Distance","perimeter","Max curv","nit"]
path='tmp/'
lst_simu=load_all_simulation(path)
values_lambda = set(map(lambda x:x.lamb, lst_simu))
newlist_simu = [[y for y in lst_simu if y.lamb==x] for x in values_lambda]

without_curv_lambda_small
with_curv_lambda_small
with_curv
output_test


In [3]:
data=[]
lst_surf=[]
for i in range(len(values_lambda)):
    #we add the ref
    partial_lst=newlist_simu[i]
    elt=partial_lst[0]
    data_dic=elt.get_data_dic_ref()
    data.append([data_dic[col_name] for col_name in column_labels])#get the item corresponding to the column label
    lst_surf.append(elt.S_ref)
    for elt in partial_lst:
        data_dic=elt.get_data_dic()
        data.append([data_dic[col_name] for col_name in column_labels])#get the item corresponding to the column label
        lst_surf.append(elt.S)
for i,row in enumerate(data):
    row.append(str(i))

In [4]:
# exchange curvature and reach
for elt in data:
   elt[10]='{:.2e}'.format(1/float(elt[10]))

In [5]:
#column_labels_latex=["Name","\lambda","Penalization","\chi_B^2","|B_{err}|_\infty","\chi_j^2","|j|_\infty","EM cost","\\text{Distance } (m)","\\text{Perimeter }(m^2)","\\text{Maximal curvature }(m^{-1})","\\text{number of iteration}"," number"]
column_labels_latex=["Name","\lambda","Penalization","\chi_B^2","|B_{err}|_\infty","\chi_j^2","|j|_\infty","EM cost","\\text{Distance } (m)","\\text{Perimeter }(m^2)","\\text{Reach }(m)","\\text{number of iteration}"," number"]
tex="\\begin{array}{"+'S'*len(column_labels_latex)+'} \n'
tex+= '&'.join(column_labels_latex)+'\\\\ \n'
for row in data:
    tex+= '&'.join(row)+'\\\\ \n'
tex+="\\end{array}"
print(tex)


\begin{array}{SSSSSSSSSSSSS} 
Name&\lambda&Penalization&\chi_B^2&|B_{err}|_\infty&\chi_j^2&|j|_\infty&EM cost&\text{Distance } (m)&\text{Perimeter }(m^2)&\text{Reach }(m)&\text{number of iteration}& number\\ 
ref&2.5e-16&None&4.80e-03&5.15e-02&1.43e+14&7.42e+06&4.06e-02&1.92e-01&5.57e+01&8.40e-02&&0\\ 
tmp/with_curv&2.5e-16&DPC&1.228e-03&3.199e-02&9.481e+13&5.986e+06&2.493e-02&1.99453e-01&5.600e+01&7.69e-02&775&1\\ 
tmp/output_test&2.5e-16&DP&1.050e-03&3.404e-02&7.361e+13&4.776e+06&1.945e-02&1.99871e-01&5.600e+01&4.33e-06&2000&2\\ 
ref&5.1e-19&None&1.44e-04&1.51e-02&4.91e+14&2.17e+07&3.94e-04&1.92e-01&5.57e+01&8.40e-02&&3\\ 
tmp/without_curv_lambda_small&5.1e-19&DP&7.159e-06&4.041e-03&1.213e+14&1.305e+07&6.901e-05&1.99987e-01&5.600e+01&8.36e-05&2000&4\\ 
tmp/with_curv_lambda_small&5.1e-19&DPC&9.048e-06&4.360e-03&1.261e+14&1.482e+07&7.337e-05&1.99983e-01&4.170e+01&7.69e-02&2000&5\\ 
\end{array}


In [6]:
from IPython.display import display, Math, Latex
display(Math(tex))

<IPython.core.display.Math object>

In [7]:
Sp_parametrization=Surface_Fourier.load_file('data/li383/plasma_surf.txt')
Sp=Surface_Fourier(Sp_parametrization,(64,64),3)

In [8]:
def plot_full_torus(lst_surf):
    from mayavi import mlab
    for surf in lst_surf:
        X,Y,Z=src.surface.surface_Fourier.expand_for_plot(surf)
        rot_tensor=src.tools.get_rot_tensor(3)
        pos=np.array([X,Y,Z])
        lpos1,lpos2,lpos3=np.einsum('sab,bij->saij',rot_tensor,pos)
        lpos_t=np.concatenate((lpos1,lpos2,lpos3),axis=2)
        lpos=np.empty(lpos_t.shape+np.array([0,0,1]))
        lpos[:,:,:-1]=lpos_t
        lpos[:,:,-1]=lpos_t[:,:,0]
        mlab.mesh(lpos[0],lpos[1],lpos[2],representation='surface',colormap='Wistia')

In [14]:
from mayavi import mlab
To_plot=[lst_surf[1],Sp]
plot_full_torus(To_plot)
plot([lst_surf[1]])

In [10]:
simu=lst_simu[1]
EM_cost,EM_dic=simu.EM.cost(simu.S)

In [11]:
j_3D=EM_dic['j_3D'].compute()
j_3D.shape

(64, 64, 3)

In [12]:
fc2.shape

NameError: name 'fc2' is not defined

In [ ]:
from src.surface.surface_Fourier import expand_for_plot
from mayavi import mlab
X,Y,Z=simu.S.X,simu.S.Y,simu.S.Z
qv=mlab.quiver3d(X[:,:], Y[:,:], Z[:,:], j_3D[:,:,0], j_3D[:,:,1], j_3D[:,:,2])
X2,Y2,Z2=expand_for_plot(simu.S)
norm=np.linalg.norm(j_3D,axis=2)
fc2=np.concatenate((norm,norm[0:1,:]),axis=0)
s = mlab.mesh(X2,Y2,Z2,representation='surface',scalars=fc2)
mlab.colorbar(s,nb_labels=4,label_fmt='%.1E',orientation='vertical')
#mlab.colorbar(s,nb_labels=4,label_fmt='%.1E',orientation='vertical')
mlab.show()

In [ ]:
S=lst_surf[1]
plot_function_on_surface(S,np.maximum(np.abs(S.principles[0]),np.abs(S.principles[1])))

In [ ]:
newlist_simu[0]+[newlist_simu[1][1],newlist_simu[1][0]]

In [ ]:
import tqdm
all_simu=newlist_simu[0]+[newlist_simu[1][1],newlist_simu[1][0]]
all_lst_dic=[]
lst_t=[]
for simu in all_simu:
    lst_dic=[]
    npts=20
    ntot_simu=len(simu.result.allvecs)
    t = [int(i*((ntot_simu-1)//npts)) for i in range(npts)]
    t.append(ntot_simu-1)
    #t+=range(1,(ntot_simu-1)//npts,2)
    t=np.sort(t)
    lst_t.append(t)
    for i in tqdm.tqdm(range(len(t))):
        elt=simu.result.allvecs[t[i]]
        lst_dic.append(simu.get_data_dic(elt))
    all_lst_dic.append(lst_dic)
    print(lst_dic[0]['Name'])

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 17]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

#print(plt.subplots(4,3))

fig, axs = plt.subplots(4,3)

for i,lst_dic in enumerate(all_lst_dic):
    ax1,ax3,ax5=axs[i]
    costsB=[float(dic['cost B']) for dic in lst_dic]
    costsJ=[float(dic['cost j']) for dic in lst_dic]

    lst_constaint_string=['Distance','perimeter','Max curv']
    lst_constaint=[[float(lst_dic[i][constaint]) for i in range(len(t))]for constaint in lst_constaint_string]
    # Create some mock data

    color_left = 'blue'
    style_left='b--'
    color_right = 'red'
    style_right='r-.'

    ax1.set_title('Main costs')
    ax1.set_xlabel('number of iterations')
    ax1.set_ylabel('$\chi_B^2 (T^2 m^2)$', color=color_left)
    #ax1.plot(t, costsB, style_left)
    ax1.scatter(lst_t[i], costsB,c=color_left,marker='^')
    ax1.tick_params(axis='y', labelcolor=color_left)
    ax1.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    ax2.set_ylabel('$\chi_j^2 (A^2)$', color=color_right)  # we already handled the x-label with ax1
    #ax2.plot(t, costsJ,style_right)
    ax2.scatter(lst_t[i], costsJ,c=color_right,marker='*')
    ax2.tick_params(axis='y', labelcolor=color_right)
    ax2.ticklabel_format(axis="y", style="sci", scilimits=(0,0))


    #Distance
    ax3.set_title('Distance and perimeter constraints')
    ax3.set_xlabel('number of iterations')
    ax3.set_ylabel('Distance($m$)', color=color_left)
    #ax3.plot(t, lst_constaint[0],style_left)
    ax3.scatter(lst_t[i], lst_constaint[0],c=color_left,marker='^')
    ax3.tick_params(axis='y', labelcolor=color_left)
    #ax3.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

    ax4 = ax3.twinx()  # instantiate a second axes that shares the same x-axis
    ax4.set_ylabel('perimeter ($m^2$)', color=color_right)  # we already handled the x-label with ax1
    #ax4.plot(t, lst_constaint[1], style_right)
    ax4.scatter(lst_t[i], lst_constaint[1],c=color_right,marker='*')
    ax4.tick_params(axis='y', labelcolor=color_right)
    #ax4.ticklabel_format(axis="y", style="sci", scilimits=(0,0))


    #perimeter  
    #ax4.set_title('perimeter constraint')

    #ax4.set_xlabel('number of iterations')
    #ax4.legend()
    #ax4.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

    #curvature 
    ax5.set_title('Curvature constraint')
    #ax5.plot(t,lst_constaint[2],style_left)
    ax5.scatter(lst_t[i],lst_constaint[2],c=color_left)
    ax5.set_yscale('log')
    ax5.set_ylabel('maximal curvature ($m^-1$)')
    ax5.set_xlabel('number of iterations')
    ax5.tick_params(axis='y')
    #ax5.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

fig.tight_layout()  # otherwise the right y-label is slightly clipped
fig.savefig('multi_plot.png')